In [1]:
import pandas as pd
import requests
import boto3

# Replace these with your AWS credentials and region
aws_access_key_id = 'YOUR_ACCESS_KEY'
aws_secret_access_key = 'YOUR_SECRET_KEY'
aws_region = 'YOUR_AWS_REGION'


In [2]:
ec2_client = boto3.client('ec2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=aws_region)

InvalidRegionError: Provided region_name 'YOUR_AWS_REGION' doesn't match a supported format.

In [ ]:
# Function to get AWS IP address ranges
def get_aws_ip_ranges():
    aws_ip_ranges = []
    try:
        response = ec2_client.describe_managed_prefix_lists()
        aws_ip_ranges = response.get('PrefixLists', [])
    except Exception as e:
        print(f"Error getting AWS IP ranges: {e}")
    return aws_ip_ranges


In [ ]:

# Function to get country information from MaxMind GeoIP2
def get_country_info(ip_address, maxmind_api_key):
    try:
        url = f"https://geoip.maxmind.com/geoip/v2.1/country/{ip_address}"
        headers = {"Authorization": f"Basic {maxmind_api_key}"}
        response = requests.get(url, headers=headers)
        country_info = response.json()
        return country_info
    except Exception as e:
        print(f"Error getting country info for {ip_address}: {e}")
        return None

# Replace 'YOUR_MAXMIND_API_KEY' with your MaxMind API key
maxmind_api_key = 'YOUR_MAXMIND_API_KEY'

# Get AWS IP address ranges
aws_ip_ranges = get_aws_ip_ranges()

# Add columns to the DataFrame for AWS Region and Country
df['AWS_Region'] = ''
df['Country'] = ''

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    ip_address = row['IP_Address']
    
    # Check if the IP address is in AWS IP ranges
    for prefix_list in aws_ip_ranges.get('PrefixLists', []):
        for entry in prefix_list.get('PrefixListEntry', []):
            if ip_address in entry.get('Cidr', ''):
                df.at[index, 'AWS_Region'] = entry.get('Regions', [''])[0]
                break
    
    # Get country information from MaxMind GeoIP2
    country_info = get_country_info(ip_address, maxmind_api_key)
    if country_info:
        df.at[index, 'Country'] = country_info.get('country', {}).get('iso_code', '')

# Display the updated DataFrame
print(df)